In [10]:
import os
import sys
import argparse
import torch
from getdist import plots, MCSamples
import getdist
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [11]:
path = os.path.realpath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, path)

In [12]:
from nnest import MCMCSampler

In [13]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [14]:
# Himmelblau
dims = 2
def loglike(z):
    z1 = z[:, 0]
    z2 = z[:, 1]
    return - (z1**2 + z2 - 11.)**2 - (z1 + z2**2 - 7.)**2

In [15]:
# Rosenbrock
#dims = 4
#def loglike(z):
#    return np.array([-sum(100.0 * (x[1:] - x[:-1] ** 2.0) ** 2.0 + (1 - x[:-1]) ** 2.0) for x in z])

In [16]:
# Gauss
#corr = 0.99
#dims = 2
#def loglike(x):
#    return multivariate_normal.logpdf(x, mean=np.zeros(dims), cov=np.eye(dims) + corr * (1 - np.eye(dims)))

In [17]:
sampler = MCMCSampler(dims, loglike, hidden_dim=64, num_layers=1, num_blocks=5, scale='constant')

Creating directory for new run logs/test/run38
[nnest.trainer] [INFO] SingleSpeedSpline(
  (net): NormalizingFlowModel(
    (flow): NormalizingFlow(
      (flows): ModuleList(
        (0): ActNorm()
        (1): Invertible1x1Conv()
        (2): NSF_CL(
          (f1): MLP(
            (net): Sequential(
              (0): Linear(in_features=1, out_features=16, bias=True)
              (1): LeakyReLU(negative_slope=0.2)
              (2): Linear(in_features=16, out_features=16, bias=True)
              (3): LeakyReLU(negative_slope=0.2)
              (4): Linear(in_features=16, out_features=16, bias=True)
              (5): LeakyReLU(negative_slope=0.2)
              (6): Linear(in_features=16, out_features=23, bias=True)
            )
          )
          (f2): MLP(
            (net): Sequential(
              (0): Linear(in_features=1, out_features=16, bias=True)
              (1): LeakyReLU(negative_slope=0.2)
              (2): Linear(in_features=16, out_features=16, bias=True)
   

In [ ]:
sampler.run(bootstrap_iters=2, jitter=0.01)

[nnest.sampler] [INFO] Alpha [0.7071]
[nnest.sampler] [INFO] Alpha [0.7071]
[nnest.sampler] [INFO] Bootstrap step [1]
[nnest.sampler] [INFO] Bootstrap step [1]
[nnest.sampler] [INFO] Getting initial samples with emcee
[nnest.sampler] [INFO] Getting initial samples with emcee
[nnest.sampler] [INFO] Mean acceptance fraction: [0.257]
[nnest.sampler] [INFO] Mean acceptance fraction: [0.257]
[nnest.trainer] [INFO] Number of training samples [2000]
[nnest.trainer] [INFO] Number of training samples [2000]
[nnest.trainer] [INFO] Training jitter [0.0100]
[nnest.trainer] [INFO] Training jitter [0.0100]
[nnest.trainer] [INFO] Epoch [1] train loss [3.0021] validation loss [2.9124]
[nnest.trainer] [INFO] Epoch [1] train loss [3.0021] validation loss [2.9124]


In [ ]:
samples = [sampler.samples[i, :, :].squeeze() for i in range(sampler.samples.shape[0])]
latent_samples = [sampler.latent_samples[i, :, :].squeeze() for i in range(sampler.latent_samples.shape[0])]
loglikes = [sampler.loglikes[i, :].squeeze() for i in range(sampler.loglikes.shape[0])]

In [ ]:
mc = MCSamples(samples=samples, loglikes=loglikes)

In [ ]:
print(mc.getEffectiveSamples())
print(mc.getMargeStats())
print(mc.getConvergeTests())

In [ ]:
g = plots.getSubplotPlotter(width_inch=8)
g.triangle_plot(mc, filled=True)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6))
ax[0].plot(samples[0][0:1000,0], samples[0][0:1000,1])
ax[1].plot(latent_samples[0][0:1000,0], latent_samples[0][0:1000,1])
plt.show()